In [ ]:
!git clone https://github.com/weharris/birdnetTest

In [ ]:
cd birdnetTest/

In [1]:
!pip3 install birdnetlib
!pip3 install tflite-runtime
!pip3 install resampy
!pip3 install ffmpeg

     -------------------------------------- 61.1/61.1 MB 630.2 kB/s eta 0:00:00
     -------------------------------------- 78.4/78.4 kB 725.1 kB/s eta 0:00:00
  Attempting uninstall: watchdog
    Found existing installation: watchdog 2.1.6
    Uninstalling watchdog-2.1.6:
      Successfully uninstalled watchdog-2.1.6


ERROR: Could not find a version that satisfies the requirement tflite-runtime (from versions: none)
ERROR: No matching distribution found for tflite-runtime


     ---------------------------------------- 3.1/3.1 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6089 sha256=38dd395092e773a5660bce741ee340f99b4c0ec7eb527fec60ea85e7021d73af
  Stored in directory: c:\users\weh90\appdata\local\pip\cache\wheels\e3\61\b9\e9c5cf3d76e5a83f61e4dc835e61ef86af884c2a48b4feeb47
Successfully built ffmpeg


In [ ]:
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
from datetime import datetime

# Load and initialize the BirdNET-Analyzer models.
analyzer = Analyzer()

Edhome = [52.911,-2.4441] # near Market Drayton year=2024, month=4, day=5
xc558223 = [57.4706, -4.163] # Highlands Scotland 2020-05-14 4am year=2020, month=5, day=14

recording = Recording(
    analyzer,
    "testamp.mp3",
    lat=xc558223[0],
    lon=-xc558223[1],
    date=datetime(year=2020, month=5, day=14), # use date or week_48
    min_conf=0.25,
)
recording.analyze()


In [ ]:
testamp = recording.detections
print(testamp)

In [ ]:
testamp[0]

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
species_count = {}
for entry in testamp:
    common_name = entry['common_name']
    if common_name in species_count:
        species_count[common_name] += 1
    else:
        species_count[common_name] = 1

df = pd.DataFrame(list(species_count.items()), columns=['Common Name', 'Count'])

In [ ]:
print(df)

In [4]:
# Plotting
plt.figure(figsize=(10, 6))  # Set the figure size
plt.bar(df['Common Name'], df['Count'], color='skyblue')  # Create a bar plot
plt.xlabel('Species')  # X-axis label
plt.ylabel('Count')  # Y-axis label
plt.title('Count of Bird Observations by Species')  # Title of the plot
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.show()  # Display the plot

NameError: name 'df' is not defined

<Figure size 1000x600 with 0 Axes>

In [ ]:
analyzer = Analyzer()

recording = Recording(
    analyzer,
    "test.mp3",
    lat=52.911,
    lon=-2.4441,
    date=datetime(year=2024, month=4, day=5), # use date or week_48
    min_conf=0.25,
)
recording.analyze()

In [ ]:
test = recording.detections
print(test)

In [6]:
# Extract confidence values for "Great Tit"
confidence_testamp = [entry['confidence'] for entry in testamp if entry['common_name'] == 'Great Tit']
confidence_test = [entry['confidence'] for entry in test if entry['common_name'] == 'Great Tit']

# Create DataFrame for plotting
data = {
    "testamp": confidence_testamp,
    "test": confidence_test
}
df = pd.DataFrame(data)

# Plotting
plt.figure(figsize=(8, 6))
df.boxplot(column=['testamp', 'test'])
plt.ylabel('Confidence')
plt.title('Confidence Levels for Great Tit Observations')
plt.show()



NameError: name 'testamp' is not defined

In [ ]:
# Perform a t-test
from scipy.stats import ttest_ind
t_stat, p_value = ttest_ind(confidence_testamp, confidence_test, equal_var=False)
print(f"T-statistic: {t_stat}, P-value: {p_value}")

## Sources
https://github.com/kahst/BirdNET-Analyzer?tab=readme-ov-file#62-setup-birdnetlib